In [ ]:
#We import some libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import uniform

import weakref 

from bayes_opt import BayesianOptimization
from root_pandas import read_root


from data_cleaning import clean_df
from KFPF_lambda_cuts import KFPF_lambda_cuts
from plot_tools import AMS, preds_prob, plot_confusion_matrix

import uproot
import gc

#To save some memory we will delete unused variables
class TestClass(object): 
    def check(self): 
        print ("object is alive!") 
    def __del__(self): 
        print ("object deleted") 
        
#to paralellize some part of the code
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(8)

In [ ]:
# We import three root files into our jupyter notebook
signal= pd.DataFrame(data=uproot.open('/home/shahid/cbmsoft/Data/PFSimplePlainTreeSignal.root:PlainTree',
                                           library='pd', decompression_executor=executor,
                                  interpretation_executor=executor).arrays(library='np',
                                decompression_executor=executor, interpretation_executor=executor))

# We only select lambda candidates
sgnal = signal[(signal['LambdaCandidates_is_signal']==1) & (signal['LambdaCandidates_mass']>1.108)
               & (signal['LambdaCandidates_mass']<1.1227)]

# Similarly for the background
background = pd.DataFrame(data=uproot.open('/home/shahid/cbmsoft/Data/PFSimplePlainTreeBackground.root:PlainTree',
                                           library='pd', decompression_executor=executor,
                                  interpretation_executor=executor).arrays(library='np',
                                decompression_executor=executor, interpretation_executor=executor))
bg = background[(background['LambdaCandidates_is_signal'] == 0)
                & ((background['LambdaCandidates_mass'] > 1.07)
                & (background['LambdaCandidates_mass'] < 1.108) | (background['LambdaCandidates_mass']>1.1227) 
                   & (background['LambdaCandidates_mass'] < 2.00))]

#delete unused variables
del signal
del background

#we also import a 10k events data set, generated using URQMD with AuAu collisions at 12AGeV
file = uproot.open('/home/shahid/cbmsoft/Data/10k_events_PFSimplePlainTree.root:PlainTree', library='pd', decompression_executor=executor,
                                  interpretation_executor=executor).arrays(library='np',decompression_executor=executor,
                                  interpretation_executor=executor)
#Call the python garbage collector to clean up things
gc.collect()
df_original= pd.DataFrame(data=file)
del file

#The labels of the columns in the df data frame are having the prefix LambdaCandidates_ so we rename them
new_labels= ['chi2geo', 'chi2primneg', 'chi2primpos', 'chi2topo', 'cosineneg',
       'cosinepos', 'cosinetopo', 'distance', 'eta', 'l', 'ldl',
       'mass', 'p', 'pT', 'phi', 'px', 'py', 'pz', 'rapidity',
             'x', 'y', 'z', 'daughter1id', 'daughter2id', 'isfrompv', 'pid', 'issignal']

sgnal.columns = new_labels
bg.columns = new_labels
df_original.columns=new_labels

# Next we clean the data using the clean_df function saved in another .py file

#Creating a new data frame and saving the results in it after cleaning of the original dfs
#Also keeping the original one
bcknd = clean_df(bg)
signal = clean_df(sgnal)

del bg
del sgnal
gc.collect()

df_clean = clean_df(df_original)
del df_original
gc.collect()

# We randomly choose our signal set of 4000 candidates
signal_selected= signal.sample(n=90000)

#background = 3 times the signal is also done randomly
background_selected = bcknd

del signal
del bcknd

#Let's combine signal and background
dfs = [signal_selected, background_selected]
df_scaled = pd.concat(dfs)
# Let's shuffle the rows randomly
df_scaled = df_scaled.sample(frac=1)
del dfs

# Creating Train and Test sets
To make machine learning algorithms more efficient on unseen data we divide our data into two sets. One set is for training the algorithm and the other is for testing the algorithm. If we don't do this then the algorithm can overfit and we will not capture the general trends in the data. 

In [ ]:
# The following columns will be used to predict whether a reconstructed candidate is a lambda particle or not
cuts = [ 'chi2primneg', 'chi2primpos', 'ldl', 'distance', 'chi2geo']


x = df_scaled[cuts].copy()

# The MC information is saved in this y variable
y =pd.DataFrame(df_scaled['issignal'], dtype='int')

## Whole set

In [ ]:
# The following columns will be used to predict whether a reconstructed candidate is a lambda particle or not
x_whole = df_clean[cuts].copy()
# The MC information is saved in this y variable
y_whole = pd.DataFrame(df_clean['issignal'], dtype='int')

# KFPF

In [ ]:
#returns a new df 
new_check_set=KFPF_lambda_cuts(df_clean)

<p style="font-family: Arial; font-size:3em;color:purple; font-style:bold"><br>XGB Boost 
<br></p><br>

## Bayesian
In order to find the best parameters of XGB for our data we use Bayesian optimization. Grid search and and random search could also do the same job but bayesian is more time efficient.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=324)

In [ ]:
dtrain = xgb.DMatrix(x_train, label = y_train)
dtest = xgb.DMatrix(x_whole, label = y_whole)
dtest1=xgb.DMatrix(x_test, label = y_test)

### Hyper parameters

*subsample* [default=1]
Subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. and this will prevent overfitting. Subsampling will occur once in every boosting iteration.
range: (0,1]

*eta* [default=0.3, alias: learning_rate]
Step size shrinkage used in update to prevents overfitting. After each boosting step, we can directly get the weights of new features, and eta shrinks the feature weights to make the boosting process more conservative.
range: [0,1]


*gamma* [default=0, alias: min_split_loss]
Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger gamma is, the more conservative the algorithm will be.
range: [0,∞]


*alpha* [default=0, alias: reg_alpha]
L1 regularization term on weights. Increasing this value will make model more conservative.

*Lasso Regression* (Least Absolute Shrinkage and Selection Operator) adds “absolute value of magnitude” of coefficient as penalty term to the loss function.

In [ ]:
#Bayesian Optimization function for xgboost
#specify the parameters you want to tune as keyword arguments
def bo_tune_xgb(max_depth, gamma, alpha, n_estimators ,learning_rate):
    params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'alpha':alpha,
              'n_estimators': n_estimators,
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'eta': 0.1,
              'eval_metric': 'auc', 'nthread' : 7}
    cv_result = xgb.cv(params, dtrain, num_boost_round=70, nfold=5)
    return  cv_result['test-auc-mean'].iloc[-1]



In [ ]:
#Invoking the Bayesian Optimizer with the specified parameters to tune
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (4, 10),
                                             'gamma': (0, 1),
                                            'alpha': (2,20),
                                             'learning_rate':(0,1),
                                             'n_estimators':(100,500)
                                            })

#performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
xgb_bo.maximize(n_iter=15, init_points=8, acq='ei')
#0.9952

# XGB models

In [ ]:
max_param = xgb_bo.max['params']
param= {'alpha': max_param['alpha'], 'gamma': max_param['gamma'], 'learning_rate': max_param['learning_rate'], 'max_depth': int(round(max_param['max_depth'],0)), 'n_estimators': int(round(max_param['n_estimators'],0)), 'objective': 'binary:logistic'}

In [ ]:
bst = xgb.train(param, dtrain)

In [ ]:
bst1= bst.predict(dtrain)

In [ ]:
bst_test = pd.DataFrame(data=bst.predict(dtest1),  columns=["xgb_preds"])
y_test=y_test.set_index(np.arange(0,bst_test.shape[0]))
bst_test['issignal']=y_test['issignal']

In [ ]:
train_best, test_best = AMS(y_train, bst1,y_test, bst_test['xgb_preds'])

In [ ]:
#The first argument should be a data frame, the second a column in it, in the form 'preds'
preds_prob(bst_test,'xgb_preds', 'issignal')

In [ ]:
#Applying XGB on the 10k events data-set
df_clean['xgb_preds'] = bst.predict(dtest)
preds_prob(df_clean,'xgb_preds', 'issignal')

## Already working model

In [ ]:
ax = xgb.plot_importance(bst)
plt.rcParams['figure.figsize'] = [8, 4]
plt.show()
ax.figure.tight_layout() 
ax.figure.savefig("hits.png")

In [ ]:
def cut_visualization(cut, range1=(1.105, 1.19), bins1= 300 ):
    mask1 = df_clean['xgb_preds']>cut
    df3=df_clean[mask1]
    
    fig, ax2 = plt.subplots(figsize=(15, 10), dpi = 200)
    color = 'tab:blue'
    ax2.hist(df_clean['mass'],bins = bins1, range=range1, facecolor='blue',alpha = 0.35, label='before selection')
    ax2.set_ylabel('Counts', fontsize = 15, color=color)
    ax2.set_xlabel('Mass in GeV', fontsize = 15)
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.legend( fontsize = 15, loc='upper left')
    
    color = 'tab:red'
    ax1 = ax2.twinx()
    ax1.hist(df3['mass'], bins = bins1, range=range1, facecolor='red',alpha = 0.35, label='Machine learning (XGB)')
    ax1.set_xlabel('Mass in GeV', fontsize = 15)
    ax1.set_ylabel('Counts ', fontsize = 15, color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.legend( fontsize = 15,loc='upper right' )
    
    
    
    plt.title("The sample's Invariant Mass with XGB (with a cut > "+str(cut)+')', fontsize = 15)
    fig.tight_layout()
    #fig.savefig("whole_sample_invmass_with_ML.png")

In [ ]:
cut_visualization(test_best)

In [ ]:
cut3 = test_best
mask1 = df_clean['xgb_preds']>test_best
df3=df_clean[mask1]
fig, axs = plt.subplots(figsize=(15, 10))

range1= (1.105, 1.14)
bins1 = 150

#xgb

df3_new = df3[df3['issignal']==1]
df3_new1 = df3[df3['issignal']==0]
df3['mass'].plot.hist(bins = bins1, range=range1, facecolor='red',alpha = 0.3,grid=True,sharey=True)
#df3_new['mass'].plot.hist(bins = 300, range=range1,facecolor='blue',alpha = 0.3,grid=True,sharey=True)
df3_new1['mass'].plot.hist(bins = bins1, range=range1,facecolor='green',alpha = 0.3,grid=True,sharey=True)
plt.legend(('XGB selected lambdas','\n False positives = \n (MC =0)\n background in \n the distribution' ), fontsize = 18, loc='upper right')
#plt.rcParams["legend.loc"] = 'upper right'
plt.title("KFPF variables + cos$\Theta_{between \ \overrightarrow{P_\Lambda} \  & \ \overrightarrow{P_{\Pi^-}}}$ + $P_T$  with a cut of %.4f "%cut3 +"on the XGB probability distribution", fontsize = 18)
plt.xlabel("Mass in $\dfrac{GeV}{c^2}$", fontsize = 18)
plt.ylabel("Counts", fontsize = 18)
axs.tick_params(labelsize=18)
fig.tight_layout()
fig.savefig("whole_sample_invmass_with_ML.png")

## Confusion Matrix

By definition a confusion matrix $C$ is such that $C_{i, j}$ is equal to the number of observations known to be in group $i$ and predicted to be in group $j$.

Thus in binary classification, the count of true positives is $C_{0,0}$, false positives is $C_{1,0}$, true negatives is $C_{1,1}$ and false negatives is $C_{0,1}$.

The following function prints and plots the confusion matrix. Normalization can be applied by setting `normalize=True`.

In [ ]:
cut1 = test_best
df_clean['xgb_preds1'] = ((df_clean['xgb_preds']>cut1)*1)
cnf_matrix = confusion_matrix(y_whole, df_clean['xgb_preds1'], labels=[1,0])
#cnf_matrix = confusion_matrix(new_check_set['issignal'], new_check_set['new_signal'], labels=[1,0])
np.set_printoptions(precision=2)
fig, axs = plt.subplots(figsize=(10, 8))
axs.yaxis.set_label_coords(-0.04,.5)
axs.xaxis.set_label_coords(0.5,-.005)
plot_confusion_matrix(cnf_matrix, classes=['signal','background'], title='Confusion Matrix for XGB for cut > '+str(cut1))
plt.savefig('confusion_matrix_extreme_gradient_boosting_whole_data.png')

<p style="font-family: Arial; font-size:3em;color:purple; font-style:bold"><br>Tree visualization
<br></p><br>

In [ ]:
#fig, ax1 = plt.subplots(figsize=(15, 10), dpi = 200)
xgb.plot_tree(bst,num_trees=10)
plt.rcParams['figure.figsize'] = [20, 40]
plt.rcParams['figure.dpi']=200
plt.show()

In [ ]:
xgb.to_graphviz(xg_reg, fmap='', num_trees=0, rankdir=None, yes_color=None, no_color=None, condition_node_params=None, leaf_node_params=None)

# Cut visualization

In [ ]:
cut3 = test_best
mask1 = df_clean['xgb_preds']>cut3
df3_base=df_clean[mask1]

In [ ]:
from matplotlib import gridspec

range1= (1.0999, 1.17)


fig, axs = plt.subplots(2, 1,figsize=(15,10), sharex=True,  gridspec_kw={'width_ratios': [10],
                           'height_ratios': [8,4]})

ns, bins, patches=axs[0].hist((df3_base['mass']),bins = 300, range=range1, facecolor='red',alpha = 0.3)
ns1, bins1, patches1=axs[0].hist((new_check_set['mass']),bins = 300, range=range1,facecolor='blue',alpha = 0.3)
#plt.xlabel("Mass in GeV", fontsize = 15)
axs[0].set_ylabel("counts", fontsize = 15)
#axs[0].grid()
axs[0].legend(('XGBoost Selected $\Lambda$s','KFPF selected $\Lambda$s'), fontsize = 15, loc='upper right')

#plt.rcParams["legend.loc"] = 'upper right'
axs[0].set_title("The lambda's Invariant Mass histogram with KFPF and XGB selection criteria on KFPF variables", fontsize = 15)
axs[0].grid()
axs[0].tick_params(axis='both', which='major', labelsize=15)
#fig.savefig("whole_sample_invmass_with_ML.png")


hist1, bin_edges1 = np.histogram(df3_base['mass'],range=(1.09, 1.17), bins=300)
hist2, bin_edges2 = np.histogram(new_check_set['mass'],range=(1.09, 1.17), bins=300)

#makes sense to have only positive values 
diff = (hist1 - hist2)
axs[1].bar(bins[:-1],     # this is what makes it comparable
        ns / ns1, # maybe check for div-by-zero!
        width=0.001)
plt.xlabel("Mass in $\dfrac{GeV}{c^2}$", fontsize = 15)
axs[1].set_ylabel("XGB / KFPF", fontsize = 15)
axs[1].grid()
axs[1].tick_params(axis='both', which='major', labelsize=15)

plt.show()
fig.tight_layout()
fig.savefig("whole_sample_invmass_with_ML.png")

## Curve Fitting

In [ ]:
from scipy.optimize import curve_fit

# Define fit function.
def fit_function(x, amplitude, mean, stddev,c,d,e):
    return (amplitude) * (np.exp(-(0.5)*((x - mean) / stddev)**2))+c+d*x+e*(x**2)
def background_function(x,c,d,e):
    return c+d*x+e*(x**2)

def signal_function( x, amplitude, mean, stddev):
    return (amplitude) * (np.exp(-(0.5)*((x - mean) / stddev)**2))
#def signal_function( x, x0, a, gam):
#    return a * (gam**2) / ( gam**2 + ( x - x0 )**2)

#def fit_function( x, x0, a, gam, c,d,e):
#    return a * (gam**2) / ( gam**2 + ( x - x0 )**2)+(c+d*x+e*(x**2))

# 3.) Generate exponential and gaussian data and histograms.
data = df3['mass']
bins = np.linspace(1.108, 1.126, 70)
data_entries_1, bins_1 = np.histogram(data, bins=bins)

# 4.) Add histograms of exponential and gaussian data.
data_entries = data_entries_1 
binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])

# 5.) Fit the function to the histogram data.
popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, p0=[600,1.115,0.001,0,0,0])
#popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, p0=[1.1156,800,0.0013,107000,-100000,800000])
print(popt)

# 6.)
# Generate enough x values to make the curves look smooth.
xspace = np.linspace(1.108, 1.126, 1000000)
fig, axs = plt.subplots(figsize=(15, 10))
# Plot the histogram and the fitted function.
plt.bar(binscenters, data_entries, width=bins[1] - bins[0], color='navy',alpha = 0.5, label=r'Histogram entries')
plt.plot(xspace, fit_function(xspace, *popt), color='darkorange', linewidth=2.5, label=r'Fitted function')


plt.plot(xspace,background_function(xspace,popt[3],popt[4],popt[5]),color='red')
plt.plot(xspace,signal_function(xspace,popt[0],popt[1],popt[2]),color='green')

# Make the plot nicer.
#plt.xlim(1.108,1.124)
plt.xlabel(r'Mass in $\frac{GeV}{c^2}$', fontsize=15)
plt.ylabel(r'Number of entries')
plt.title(r'Lamda baryon')
plt.legend(loc='best')
plt.show()
plt.clf()
fig.savefig('hists.png')

In [ ]:
from scipy.integrate import quad
def integrand( x, x0, a, gam):
    return a * (gam**2) / ( gam**2 + ( x - x0 )**2)

x0 = popt[0]
a = popt[1]
gam = popt[2]
I_sig = quad(integrand, popt[0]-5*popt[2], popt[0]+5*popt[2] , args=(x0, a, gam))
I_sig

In [ ]:
def integrand(x,c,d,e):
    return c+d*x+e*(x**2)


c = popt[3]
d = popt[4]
e = popt[5]
I = quad(integrand, popt[0]-5*popt[2], popt[0]+5*popt[2] , args=(c,d,e))
I_sig[0]/abs(I[0])

In [ ]:
def integrand(x, amplitude, mean, stddev,c,d,e):
    return (amplitude) * (np.exp(-(0.5)*((x - mean) / stddev)**2))+c+d*x+e*(x**2)

amplitude = popt[0]
mean = popt[1]
stddev = popt[2]
c = popt[3]
d = popt[4]
e = popt[5]
I = quad(integrand, popt[1]-5*popt[2], popt[1]+5*popt[2] , args=(amplitude,mean,stddev,c,d,e))
I_sig[0]/(np.sqrt(I[0]))

In [ ]:
from scipy.optimize import curve_fit

# Define fit function.
def fit_function(x, amp, mean, std,c,d,e):
    return (amp/(np.sqrt(2*np.pi) * std)) * (np.exp(-(0.5)*((x - mean) / std)**2))+c+d*x+e*(x**2)
def background_function(x,c,d,e):
    return c+d*x+e*(x**2)

def signal_function( x, amp, mean, std):
    return (amp/(np.sqrt(2*np.pi) * std)) * (np.exp(-(0.5)*((x - mean) / std)**2))
#def signal_function( x, a,mean, gam):
#    return a * (gam**2) / ( gam**2 + ( x - mean )**2)

#def fit_function( x, a, mean, gam, c,d,e):
#    return a * (gam**2) / ( gam**2 + ( x - mean )**2)+(c+d*x)++e*(x**2)

# 3.) Generate exponential and gaussian data and histograms.
data = df3['mass']
bins = np.linspace(1.105, 1.126, 70)
data_entries_1, bins_1 = np.histogram(data, bins=bins)

# 4.) Add histograms of exponential and gaussian data.
data_entries = data_entries_1 
binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])

# 5.) Fit the function to the histogram data.
popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, p0=[600,1.115,0.001,0,0,0])
#popt, pcov = curve_fit(fit_function, xdata=binscenters, ydata=data_entries, p0=[1,1.1156,0.001,0,0,0])
print(popt)

# 6.)
# Generate enough x values to make the curves look smooth.
xspace = np.linspace(1.105, 1.126, 1000000)
fig, axs = plt.subplots(figsize=(15, 10))
# Plot the histogram and the fitted function.
plt.bar(binscenters, data_entries, width=bins[1] - bins[0], color='navy',alpha = 0.5, label=r'Histogram entries')
plt.plot(xspace, fit_function(xspace, *popt), color='darkorange', linewidth=2.5, label=r'Fitted function')


plt.plot(xspace,background_function(xspace,popt[3],popt[4],popt[5]),color='red')
plt.plot(xspace,signal_function(xspace,popt[0],popt[1],popt[2]),color='green')

# Make the plot nicer.
#plt.xlim(1.108,1.124)
plt.xlabel(r'Mass in $\frac{GeV}{c^2}$', fontsize=15)
plt.ylabel(r'Number of entries')
plt.title(r'Lamda baryon')
plt.legend(loc='best')
plt.show()
plt.clf()
fig.savefig('hists.png')
#pcov

In [ ]:
result.values

In [ ]:
x=binscenters, 
y=data_entries

#def gaussian(x, amp, cen, wid, c,d,e):
#    """1-d gaussian: gaussian(x, amp, cen, wid)"""
#    return ((amp / (np.sqrt(2*np.pi) * wid)) * np.exp(-(x-cen)**2 / (2*wid**2)))+c+d*x+e*(x**2)


gmodel = Model(fit_function)
result = gmodel.fit(y, x=x, amp=3, mean=1.115, std=0.0009, c=20, d=10, e=-10,  method='bfgs')
#result = gmodel.fit(y, x=x, a=900, mean=1.1157, gam=0.0009, c=-100, d=100, e=10, method='cg')

print(result.fit_report())

fig, axs = plt.subplots(figsize=(15, 10))
plt.bar(binscenters, data_entries, width=bins[1] - bins[0], color='navy',alpha = 0.5, label=r'Histogram entries')
#plt.plot(x, result.init_fit, 'k--', label='initial fit')
#plt.plot(x, result.best_fit, 'r-', label='best fit')
plt.plot(xspace, fit_function(xspace, result.values['amp'],result.values['mean'],result.values['std']
                          ,result.values['c'],result.values['d'],result.values['e']))

plt.plot(xspace,background_function(xspace,result.values['c'],result.values['d'],result.values['e']),color='red')
plt.plot(xspace,signal_function(xspace,result.values['amp'],result.values['mean'],result.values['std'])
         ,color='green')
#plt.legend(loc='best')
#plt.show()
# <end examp